In [1]:
import pandas as pd
import nltk
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [2]:
data_path = '/Users/shiwei/Desktop/研究论文/一带一路ERGM数据分析/all_BeltandRoad_tweets.csv'

df = pd.read_csv(data_path, encoding = 'utf8')

/var/folders/wq/891pj93x7r99n97_2h5wxzzw0000gn/T/ipykernel_5001/3107322003.py:3: DtypeWarning: Columns (3,11,19,39,42,43) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_path, encoding = 'utf8')


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1248546 entries, 0 to 1248545
Data columns (total 56 columns):
 #   Column                                 Non-Null Count    Dtype  
---  ------                                 --------------    -----  
 0   Unnamed: 0                             1248546 non-null  int64  
 1   in_reply_to_username                   75620 non-null    object 
 2   retweeted_username                     755618 non-null   object 
 3   quoted_username                        18517 non-null    object 
 4   created_at                             1248546 non-null  object 
 5   text                                   1248546 non-null  object 
 6   public_metrics.impression_count        1248546 non-null  int64  
 7   public_metrics.reply_count             1248546 non-null  int64  
 8   public_metrics.retweet_count           1248546 non-null  int64  
 9   public_metrics.quote_count             1248546 non-null  int64  
 10  public_metrics.like_count              124

In [4]:
selected_columns = df[['text',
                       'public_metrics.reply_count',
                       'public_metrics.retweet_count',
                       'public_metrics.quote_count',
                       'public_metrics.like_count']]

df = pd.DataFrame(selected_columns)

In [5]:
numeric_columns = df.select_dtypes(include = ['number'])  # 选择数值型列
df['means'] = numeric_columns.mean(axis = 1)
df.drop(numeric_columns.columns, axis = 1, inplace = True)

In [6]:
df

,text,means
0,The new silk road. What are your thoughts on t...,0.0
1,The new Silk Road โ€“ railways from China to W...,0.0
2,The New Silk Road: China-Europe Railways http:...,0.0
3,One day I'm going to check this out. The New ...,0.0
4,Check out the latest edition of โ€JICA Worldโ€...,0.5
...,...,...
1248541,.@Matt_Schrader_ and @JMichaelCole1 discuss Ch...,4.0
1248542,"this says ""belt and road"" v1.0 is over . hadn'...",1.0
1248543,https://t.co/wVUdQPQF5k\n\n“If the government ...,14.0
1248544,.@Matt_Schrader_ and @JMichaelCole1 discuss Ch...,4.0


In [7]:
analyzer = SentimentIntensityAnalyzer() #加载情感分析函数

In [8]:
#开始情感分析
senti_list = []
for sentence in df['text']:
    vs = analyzer.polarity_scores(sentence)
    senti_list.append(vs)
    #print("{:-<65} {}".format(sentence, str(vs)))
    #print("*"*100)

In [9]:
#senti_list列表中存储的是一个一个的字典，将字典转换成DataFrame格式的df_senti
df_senti = pd.DataFrame(senti_list)

In [10]:
df_senti #查看数据结构

,neg,neu,pos,compound
0,0.000,1.000,0.000,0.0000
1,0.000,1.000,0.000,0.0000
2,0.000,1.000,0.000,0.0000
3,0.000,1.000,0.000,0.0000
4,0.000,1.000,0.000,0.0000
...,...,...,...,...
1248541,0.000,1.000,0.000,0.0000
1248542,0.122,0.878,0.000,-0.3612
1248543,0.094,0.766,0.140,0.0818
1248544,0.000,1.000,0.000,0.0000


In [11]:
df['sentiment'] = df_senti['compound']

In [12]:
df

,text,means,sentiment
0,The new silk road. What are your thoughts on t...,0.0,0.0000
1,The new Silk Road โ€“ railways from China to W...,0.0,0.0000
2,The New Silk Road: China-Europe Railways http:...,0.0,0.0000
3,One day I'm going to check this out. The New ...,0.0,0.0000
4,Check out the latest edition of โ€JICA Worldโ€...,0.5,0.0000
...,...,...,...
1248541,.@Matt_Schrader_ and @JMichaelCole1 discuss Ch...,4.0,0.0000
1248542,"this says ""belt and road"" v1.0 is over . hadn'...",1.0,-0.3612
1248543,https://t.co/wVUdQPQF5k\n\n“If the government ...,14.0,0.0818
1248544,.@Matt_Schrader_ and @JMichaelCole1 discuss Ch...,4.0,0.0000


In [13]:
df["text"] = df["text"].str.lower()  # 统一为小写字母

In [14]:
# 导入nltk的分词工具word_tokenize和nltk的停用词表stopwords，需要自行下载哦
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

stopwords_list = stopwords.words("english")

In [15]:
# 定义一个函数，处理Twitter文本中的特殊符号：@xxx、&xxx、网址、多余的空格，最后只保留数字和字母
def clean_specialtext(tweet):
    tweet = re.sub(r"@([^\s:]+)", " ", tweet)  # 去除@提及内容
    tweet = re.sub(r"&([^\s:]+)", " ", tweet)  # 去除&及之后的内容
    tweet = re.sub(r"[a-zA-z]+://[^\s]*", " ", tweet)  # 去除网址
    tweet = re.sub(r"[^0-9A-Za-z]", " ", tweet)  # 仅保留数字及字母
    tweet = re.sub(r"(^\s*)|(\s*$)", "", tweet)  # 去除字符串前后空格
    # tweet = re.sub(r'[^0-9A-Za-z]', ' ', tweet)
    return tweet

In [16]:
df["text"] = df["text"].apply(lambda x: clean_specialtext(x))

In [17]:
df

,text,means,sentiment
0,the new silk road what are your thoughts on t...,0.0,0.0000
1,the new silk road railways from china to w...,0.0,0.0000
2,the new silk road china europe railways,0.0,0.0000
3,one day i m going to check this out the new ...,0.0,0.0000
4,check out the latest edition of jica world ...,0.5,0.0000
...,...,...,...
1248541,and discuss china s revamped belt and road i...,4.0,0.0000
1248542,this says belt and road v1 0 is over hadn ...,1.0,-0.3612
1248543,the government is not seriously tackling the p...,14.0,0.0818
1248544,and discuss china s revamped belt and road i...,4.0,0.0000


In [18]:
#导入nltk的词性标注工具pos_tag、词性属性工具wordnet和词形还原工具WordNetLemmatizer
from nltk.stem import WordNetLemmatizer
from nltk.tag import pos_tag
from nltk.corpus import wordnet
lemmatizer = WordNetLemmatizer()

In [19]:
# 这一步是为了更精确词形还原，对词性进行标注，精确的词性对词形还原很有必要
# 参考https://stackoverflow.com/questions/61982023/using-wordnetlemmatizer-lemmatize-with-pos-tags-throws-keyerror
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith("J"):
        return wordnet.ADJ
    elif treebank_tag.startswith("V"):
        return wordnet.VERB
    elif treebank_tag.startswith("N"):
        return wordnet.NOUN
    elif treebank_tag.startswith("R"):
        return wordnet.ADV
    else:
        return None

In [20]:
# 定义去除停用词和词形还原函数，每一步都是有顺序的，不能乱

def clean_stopwords_Lemmatization(tweet):
    tweet = word_tokenize(tweet)
    # print(tweet)
    # print("*"*100)
    tweet = [word for word in tweet if word not in stopwords_list]  # 去除停用词
    tweet = pos_tag(tweet)  # 词性标注
    # 词性标注好之后，列表里变成了（词，词性）的元组形式，word[0]是词的位置，对word[0]进行词形还原，依据是上面的函数和word[1]
    tweet = [
        lemmatizer.lemmatize(word[0], pos=get_wordnet_pos(word[1]))
        if get_wordnet_pos(word[1])
        else word[0]
        for word in tweet
    ]
    tweet = [word for word in tweet if len(word) > 2]  # 丢弃长度小于等于2的字符串
    tweet = " ".join(tweet)  # 将tweet列表中的字符串重新拼接起来，用空格分隔
    # print(tweet)
    # print("#"*100)
    return tweet

In [21]:
df['text'] = df['text'].apply(lambda x: clean_stopwords_Lemmatization(x))

In [22]:
df

,text,means,sentiment
0,new silk road thought trans siberian railway t...,0.0,0.0000
1,new silk road railways china western europe,0.0,0.0000
2,new silk road china europe railway,0.0,0.0000
3,one day check new silk road via,0.0,0.0000
4,check late edition jica world feature jica eff...,0.5,0.0000
...,...,...,...
1248541,discuss china revamp belt road initiative expl...,4.0,0.0000
1248542,say belt road heard china medium coverage larg...,1.0,-0.3612
1248543,government seriously tackle problem society in...,14.0,0.0818
1248544,discuss china revamp belt road initiative expl...,4.0,0.0000


In [23]:
df.to_csv('/Users/shiwei/Desktop/研究论文/一带一路ERGM数据分析/All_Cleaned_BRI_Tweets.csv', encoding = 'utf8')